***GENERATED CODE FOR data PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
from hdfs3 import HDFileSystem
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs = HDFileSystem(
            host=eval(config)['host'], port=eval(config)['port'])
        with hdfs.open(eval(config)['url']) as f:
            df = pd.read_csv(f, error_bad_lines=False)
        df = spark.createDataFrame(dfPd)
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.functions import col, when
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                                         meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***OPERATION FUNCTIONS***

In [ ]:
import pyspark
from dask.dataframe import from_pandas
import json


def replaceValues(df, functionsData, applyOn):
    for columnData in applyOn:
        for functionData in functionsData:
            if functionData['replaceType'] == 'value':
                if functionData['replaceValueType'] == 'min':
                    minValue = df[columnData['columnName']].min().compute()
                    df[columnData['columnName']] = df[columnData['columnName']].replace(
                        toReplace, minValue)
                elif functionData['replaceValueType'] == 'max':
                    maxValue = df[columnData['columnName']].max().compute()
                    df[columnData['columnName']] = df[columnData['columnName']].replace(
                        toReplace, maxValue)
                else:
                    df[columnData['columnName']] = df[columnData['columnName']].replace(
                        toReplace, functionData['ReplaceWith'])
            elif functionData['replaceType'] == 'none':
                if functionData['replaceValueType'] == 'min':
                    minValue = df[columnData['columnName']].min().compute()
                    df[columnData['columnName']
                       ] = df[columnData['columnName']].replace("", minValue)
                    df[columnData['columnName']
                       ] = df[columnData['columnName']].fillna(minValue)
                elif functionData['replaceValueType'] == 'max':
                    maxValue = df[columnData['columnName']].max().compute()
                    df[columnData['columnName']
                       ] = df[columnData['columnName']].replace("", maxValue)
                    df[columnData['columnName']
                       ] = df[columnData['columnName']].fillna(maxValue)
                else:
                    df[columnData['columnName']] = df[columnData['columnName']].replace(
                        "", functionData['ReplaceWith'])
                    df[columnData['columnName']] = df[columnData['columnName']].fillna(
                        functionData['ReplaceWith'])
    return df


def renameColumns(df, functionsData, applyOn):
    for functionData in functionsData:
        df = df.rename(
            columns={functionData['oldName']: functionData['newName']})
    return df


def runDataCleansing(sparkDf, spark, config):
    configObj = json.loads(config)
    sparkDf.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = from_pandas((sparkDf.toPandas()), npartitions=5)
    functionList = configObj['functionsApplied']
    Data_Cleansing_Methods = {"replaceBy": replaceValues,
                              "formula": calculateFormula,
                              "aggregate": aggregation,
                              "converttostringtype": changeToString,
                              "editname": renameColumns}
    for function in functionList:
        function['functionName']
        df = Data_Cleansing_Methods[function['functionName']](df, function['functionsData'],
                                                              function['applyOn'])
    sparkDf = spark.createDataFrame(df.compute())

    display(sparkDf.limit(2).toPandas())
    return sparkDf


***AUTOML FUNCTIONS***

In [ ]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
import pyspark


def functionClassification(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = (sparkDF.toPandas())
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTClassifier(verbosity=3, n_jobs=-1, generations=10, max_time_mins=5,
                               population_size=15)
    tpotModel.fit(X_train, y_train)
    display(" Accuracy of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################from pyspark.sql import SparkSession
spark = SparkSession.builder.master('spark://0.0.0.0:0000').getOrCreate()
#%run dataHooks.ipynb
try:
	#sourcePreExecutionHook()

	fetalhealth = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/fetal_health.csv', 'filename': 'fetal_health.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'dbfs_token': '', 'dbfs_domain': '', 'is_header': 'Use Header Line', 'server_url': '/numtraPlatform/NumtraPlatformV2/uploads/platform/', 'results_url': 'http://3.140.115.35:4040/api/read/hdfs'}")

except Exception as ex: 
	logging.error(ex)


***TRANSFORMING DATAFRAME***

In [ ]:
#%run dataHooks.ipynb
try:
	#transformationPreExecutionHook()

	autofe = TransformationMain.run(fetalhealth,json.dumps( {"FE": [{"transformationsData": [{"transformation_label": "novalue"}], "feature": "baseline value", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "42", "mean": "133.69", "stddev": "10.16", "min": "116", "max": "158", "missing": "0"}, "updatedLabel": "baseline value"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "accelerations", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "42", "mean": "0.0", "stddev": "0.0", "min": "0.0", "max": "0.014", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "accelerations"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "fetal_movement", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "42", "mean": "0.01", "stddev": "0.01", "min": "0.0", "max": "0.048", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "fetal_movement"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "uterine_contractions", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "42", "mean": "0.0", "stddev": "0.0", "min": "0.0", "max": "0.013", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "uterine_contractions"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "light_decelerations", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "42", "mean": "0.0", "stddev": "0.0", "min": "0.0", "max": "0.015", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "light_decelerations"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "severe_decelerations", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "42", "mean": "0.0", "stddev": "0.0", "min": "0", "max": "0", "missing": "0"}, "updatedLabel": "severe_decelerations"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "prolongued_decelerations", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "42", "mean": "0.0", "stddev": "0.0", "min": "0.0", "max": "0.003", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "prolongued_decelerations"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "abnormal_short_term_variability", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "42", "mean": "48.33", "stddev": "16.0", "min": "22", "max": "84", "missing": "0"}, "updatedLabel": "abnormal_short_term_varia..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "mean_value_of_short_term_variability", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "42", "mean": "1.31", "stddev": "1.32", "min": "0.2", "max": "6.3", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "mean_value_of_short_term_..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "percentage_of_time_with_abnormal_long_term_variability", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "42", "mean": "12.6", "stddev": "19.8", "min": "0", "max": "81", "missing": "0"}, "updatedLabel": "percentage_of_time_with_a..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "mean_value_of_long_term_variability", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "42", "mean": "9.31", "stddev": "5.85", "min": "0.0", "max": "27.6", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "mean_value_of_long_term_v..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "histogram_width", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "42", "mean": "67.5", "stddev": "43.05", "min": "10", "max": "150", "missing": "0"}, "updatedLabel": "histogram_width"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "histogram_min", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "42", "mean": "97.5", "stddev": "33.8", "min": "50", "max": "153", "missing": "0"}, "updatedLabel": "histogram_min"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "histogram_max", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "42", "mean": "165.0", "stddev": "20.33", "min": "131", "max": "200", "missing": "0"}, "updatedLabel": "histogram_max"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "histogram_number_of_peaks", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "42", "mean": "3.83", "stddev": "2.89", "min": "0", "max": "11", "missing": "0"}, "updatedLabel": "histogram_number_of_peaks"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "histogram_number_of_zeroes", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "42", "mean": "0.33", "stddev": "0.69", "min": "0", "max": "3", "missing": "0"}, "updatedLabel": "histogram_number_of_zeroe..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "histogram_mode", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "42", "mean": "139.67", "stddev": "18.65", "min": "71", "max": "187", "missing": "0"}, "updatedLabel": "histogram_mode"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "histogram_mean", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "42", "mean": "138.36", "stddev": "13.97", "min": "107", "max": "178", "missing": "0"}, "updatedLabel": "histogram_mean"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "histogram_median", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "42", "mean": "140.07", "stddev": "13.88", "min": "106", "max": "180", "missing": "0"}, "updatedLabel": "histogram_median"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "histogram_variance", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "42", "mean": "18.43", "stddev": "40.19", "min": "0", "max": "215", "missing": "0"}, "updatedLabel": "histogram_variance"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "histogram_tendency", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "42", "mean": "0.29", "stddev": "0.6", "min": "-1", "max": "1", "missing": "0"}, "updatedLabel": "histogram_tendency"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "fetal_health", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "42", "mean": "1.29", "stddev": "0.55", "min": "1", "max": "3", "missing": "0"}, "updatedLabel": "fetal_health"}]}))

	#transformationPostExecutionHook(autofe)

except Exception as ex: 
	logging.error(ex)


***PERFORMING OPERATIONS***

In [ ]:
#%run dataHooks.ipynb
try:
	#operationPreExecutionHook()

datapreparation = runDataCleansing(autofe,spark,json.dumps( {"url": "/FileStore/platform/uploadedSourceFiles/fetal_health.csv", "source_attributes": {}, "DataPrepFile": "/FileStore/platform/uploadedSourceFiles/fetal_health.csv", "data_source": "localfiles", "startListenerOnly": 1, "dateColumnNames": [], "FilePath": "/FileStore/platform/extra/613b249c76ca36277e2e6eee1631268620/0part.csv", "requestRatio": 2.0, "totalRows": 799, "BasicStats": {"missingValues": 0.0, "numberOfColumns": 23, "numberOfRows": 799, "duplicateRowCount": 10, "stats": [{"column": "baseline value", "alias": "baseline value", "generated": 0, "type": "numeric", "max": 159.0, "min": 114.0, "mean": 135.20025031289111, "missing": 0.0, "stddev": 10.52}, {"column": "accelerations", "alias": "accelerations", "generated": 0, "type": "real", "max": 0.0, "min": 0.0, "mean": 0.0, "missing": 0.0, "stddev": 0.0}, {"column": "fetal_movement", "alias": "fetal_movement", "generated": 0, "type": "real", "max": 0.0, "min": 0.0, "mean": 0.0, "missing": 0.0, "stddev": 0.0}, {"column": "uterine_contractions", "alias": "uterine_contractions", "generated": 0, "type": "real", "max": 0.0, "min": 0.0, "mean": 0.0, "missing": 0.0, "stddev": 0.0}, {"column": "light_decelerations", "alias": "light_decelerations", "generated": 0, "type": "real", "max": 0.0, "min": 0.0, "mean": 0.0, "missing": 0.0, "stddev": 0.0}, {"column": "severe_decelerations", "alias": "severe_decelerations", "generated": 0, "type": "numeric", "max": 0.0, "min": 0.0, "mean": 0.0, "missing": 0.0, "stddev": 0.0}, {"column": "prolongued_decelerations", "alias": "prolongued_decelerations", "generated": 0, "type": "real", "max": 0.0, "min": 0.0, "mean": 0.0, "missing": 0.0, "stddev": 0.0}, {"column": "abnormal_short_term_variability", "alias": "abnormal_short_term_variability", "generated": 0, "type": "numeric", "max": 87.0, "min": 12.0, "mean": 51.01251564455569, "missing": 0.0, "stddev": 16.98}, {"column": "mean_value_of_short_term_variability", "alias": "mean_value_of_short_term_variability", "generated": 0, "type": "real", "max": 7.0, "min": 0.0, "mean": 0.7246558197747184, "missing": 0.0, "stddev": 1.1}, {"column": "percentage_of_time_with_abnormal_long_term_variability", "alias": "percentage_of_time_with_abnormal_long_term_variability", "generated": 0, "type": "numeric", "max": 91.0, "min": 0.0, "mean": 15.538172715894868, "missing": 0.0, "stddev": 22.67}, {"column": "mean_value_of_long_term_variability", "alias": "mean_value_of_long_term_variability", "generated": 0, "type": "real", "max": 50.0, "min": 0.0, "mean": 9.191489361702128, "missing": 0.0, "stddev": 6.15}, {"column": "histogram_width", "alias": "histogram_width", "generated": 0, "type": "numeric", "max": 150.0, "min": 3.0, "mean": 74.41426783479349, "missing": 0.0, "stddev": 45.49}, {"column": "histogram_min", "alias": "histogram_min", "generated": 0, "type": "numeric", "max": 159.0, "min": 50.0, "mean": 91.75344180225282, "missing": 0.0, "stddev": 34.03}, {"column": "histogram_max", "alias": "histogram_max", "generated": 0, "type": "numeric", "max": 200.0, "min": 126.0, "mean": 166.1677096370463, "missing": 0.0, "stddev": 19.99}, {"column": "histogram_number_of_peaks", "alias": "histogram_number_of_peaks", "generated": 0, "type": "numeric", "max": 18.0, "min": 0.0, "mean": 4.394242803504381, "missing": 0.0, "stddev": 3.43}, {"column": "histogram_number_of_zeroes", "alias": "histogram_number_of_zeroes", "generated": 0, "type": "numeric", "max": 10.0, "min": 0.0, "mean": 0.31289111389236546, "missing": 0.0, "stddev": 0.82}, {"column": "histogram_mode", "alias": "histogram_mode", "generated": 0, "type": "numeric", "max": 187.0, "min": 71.0, "mean": 140.35043804755944, "missing": 0.0, "stddev": 14.94}, {"column": "histogram_mean", "alias": "histogram_mean", "generated": 0, "type": "numeric", "max": 182.0, "min": 87.0, "mean": 139.0650813516896, "missing": 0.0, "stddev": 13.21}, {"column": "histogram_median", "alias": "histogram_median", "generated": 0, "type": "numeric", "max": 186.0, "min": 77.0, "mean": 141.09887359198999, "missing": 0.0, "stddev": 13.39}, {"column": "histogram_variance", "alias": "histogram_variance", "generated": 0, "type": "numeric", "max": 215.0, "min": 0.0, "mean": 14.893617021276595, "missing": 0.0, "stddev": 26.2}, {"column": "histogram_tendency", "alias": "histogram_tendency", "generated": 0, "type": "numeric", "max": 1.0, "min": -1.0, "mean": 0.3954943679599499, "missing": 0.0, "stddev": 0.61}, {"column": "fetal_health", "alias": "fetal_health", "generated": 0, "type": "numeric", "max": 3.0, "min": 1.0, "mean": 1.4831038798498122, "missing": 0.0, "stddev": 0.67}, {"column": "test col", "alias": "test col", "generated": 1, "type": "real", "max": 7.0, "min": 0.0, "mean": 0.7246558197747184, "missing": 0.0, "stddev": 1.1}]}, "predictionPowerScore": [{"abnormal_short_term_variability": 1.0, "accelerations": 0.0, "baseline value": 0.167757083, "fetal_health": 0.3216081289, "fetal_movement": 0.0, "histogram_max": 0.1409571914, "histogram_mean": 0.0871483281, "histogram_median": 0.0728893667, "histogram_min": 0.2314250088, "histogram_mode": 0.1487515837, "histogram_number_of_peaks": 0.1689027789, "histogram_number_of_zeroes": 0.0564054057, "histogram_tendency": 0.0, "histogram_variance": 0.2782374746, "histogram_width": 0.190974013, "light_decelerations": 0.0, "mean_value_of_long_term_variability": 0.0937509854, "mean_value_of_short_term_variability": 0.3709816406, "percentage_of_time_with_abnormal_long_term_variability": 0.3343347587, "prolongued_decelerations": 0.0, "severe_decelerations": 0.0, "test col": 0.3709816406, "uterine_contractions": 0.0}, {"abnormal_short_term_variability": 0.0, "accelerations": 1.0, "baseline value": 0.0, "fetal_health": 0.0, "fetal_movement": 0.0, "histogram_max": 0.0, "histogram_mean": 0.0, "histogram_median": 0.0, "histogram_min": 0.0, "histogram_mode": 0.0, "histogram_number_of_peaks": 0.0, "histogram_number_of_zeroes": 0.0, "histogram_tendency": 0.0, "histogram_variance": 0.0, "histogram_width": 0.0, "light_decelerations": 0.0, "mean_value_of_long_term_variability": 0.0, "mean_value_of_short_term_variability": 0.0, "percentage_of_time_with_abnormal_long_term_variability": 0.0, "prolongued_decelerations": 0.0, "severe_decelerations": 0.0, "test col": 0.0, "uterine_contractions": 0.0}, {"abnormal_short_term_variability": 0.0808218359, "accelerations": 0.0, "baseline value": 1.0, "fetal_health": 0.1121624022, "fetal_movement": 0.0, "histogram_max": 0.1226316204, "histogram_mean": 0.5079794425, "histogram_median": 0.5640636531, "histogram_min": 0.2659805604, "histogram_mode": 0.5724633104, "histogram_number_of_peaks": 0.0503206864, "histogram_number_of_zeroes": 0.0, "histogram_tendency": 0.0347083177, "histogram_variance": 0.0, "histogram_width": 0.0107683857, "light_decelerations": 0.0, "mean_value_of_long_term_variability": 0.0, "mean_value_of_short_term_variability": 0.0647886627, "percentage_of_time_with_abnormal_long_term_variability": 0.0026177752, "prolongued_decelerations": 0.0, "severe_decelerations": 0.0, "test col": 0.0647886627, "uterine_contractions": 0.0}, {"abnormal_short_term_variability": 0.4848863974, "accelerations": 0.0, "baseline value": 0.0309154787, "fetal_health": 1.0, "fetal_movement": 0.0, "histogram_max": 0.0, "histogram_mean": 0.0, "histogram_median": 0.0, "histogram_min": 0.0915658847, "histogram_mode": 0.0, "histogram_number_of_peaks": 0.0, "histogram_number_of_zeroes": 0.0, "histogram_tendency": 0.0, "histogram_variance": 0.1265452759, "histogram_width": 0.1345796935, "light_decelerations": 0.0, "mean_value_of_long_term_variability": 0.0, "mean_value_of_short_term_variability": 0.039215239, "percentage_of_time_with_abnormal_long_term_variability": 0.2741755231, "prolongued_decelerations": 0.0, "severe_decelerations": 0.0, "test col": 0.039215239, "uterine_contractions": 0.0}, {"abnormal_short_term_variability": 0.0, "accelerations": 0.0, "baseline value": 0.0, "fetal_health": 0.0, "fetal_movement": 1.0, "histogram_max": 0.0, "histogram_mean": 0.0, "histogram_median": 0.0, "histogram_min": 0.0, "histogram_mode": 0.0, "histogram_number_of_peaks": 0.0, "histogram_number_of_zeroes": 0.0, "histogram_tendency": 0.0, "histogram_variance": 0.0, "histogram_width": 0.0, "light_decelerations": 0.0, "mean_value_of_long_term_variability": 0.0, "mean_value_of_short_term_variability": 0.0, "percentage_of_time_with_abnormal_long_term_variability": 0.0, "prolongued_decelerations": 0.0, "severe_decelerations": 0.0, "test col": 0.0, "uterine_contractions": 0.0}, {"abnormal_short_term_variability": 0.1016802909, "accelerations": 0.0, "baseline value": 0.0900116545, "fetal_health": 0.0537917557, "fetal_movement": 0.0, "histogram_max": 1.0, "histogram_mean": 0.0816219092, "histogram_median": 0.1075776324, "histogram_min": 0.1705007863, "histogram_mode": 0.1066781505, "histogram_number_of_peaks": 0.2270828761, "histogram_number_of_zeroes": 0.0802207397, "histogram_tendency": 0.0061420095, "histogram_variance": 0.2411162761, "histogram_width": 0.4475275044, "light_decelerations": 0.0, "mean_value_of_long_term_variability": 0.0526295787, "mean_value_of_short_term_variability": 0.2230145346, "percentage_of_time_with_abnormal_long_term_variability": 0.1221914184, "prolongued_decelerations": 0.0, "severe_decelerations": 0.0, "test col": 0.2230145346, "uterine_contractions": 0.0}, {"abnormal_short_term_variability": 0.0384280388, "accelerations": 0.0, "baseline value": 0.5028757822, "fetal_health": 0.0378407144, "fetal_movement": 0.0, "histogram_max": 0.1363914219, "histogram_mean": 1.0, "histogram_median": 0.8378375125, "histogram_min": 0.1840831222, "histogram_mode": 0.676577149, "histogram_number_of_peaks": 0.0290235125, "histogram_number_of_zeroes": 0.0, "histogram_tendency": 0.0540925929, "histogram_variance": 0.0, "histogram_width": 0.0136255738, "light_decelerations": 0.0, "mean_value_of_long_term_variability": 0.0, "mean_value_of_short_term_variability": 0.036400408, "percentage_of_time_with_abnormal_long_term_variability": 0.0, "prolongued_decelerations": 0.0, "severe_decelerations": 0.0, "test col": 0.036400408, "uterine_contractions": 0.0}, {"abnormal_short_term_variability": 0.0246162577, "accelerations": 0.0, "baseline value": 0.5376715953, "fetal_health": 0.0405624006, "fetal_movement": 0.0, "histogram_max": 0.1618632978, "histogram_mean": 0.8367218648, "histogram_median": 1.0, "histogram_min": 0.178524509, "histogram_mode": 0.7757197594, "histogram_number_of_peaks": 0.0323588709, "histogram_number_of_zeroes": 0.0, "histogram_tendency": 0.0650114994, "histogram_variance": 0.0, "histogram_width": 0.0, "light_decelerations": 0.0, "mean_value_of_long_term_variability": 0.0, "mean_value_of_short_term_variability": 0.0263923974, "percentage_of_time_with_abnormal_long_term_variability": 0.0, "prolongued_decelerations": 0.0, "severe_decelerations": 0.0, "test col": 0.0263923974, "uterine_contractions": 0.0}, {"abnormal_short_term_variability": 0.1994233215, "accelerations": 0.0, "baseline value": 0.1352855258, "fetal_health": 0.1351091149, "fetal_movement": 0.0, "histogram_max": 0.1673852534, "histogram_mean": 0.0623505819, "histogram_median": 0.0529604241, "histogram_min": 1.0, "histogram_mode": 0.1087344749, "histogram_number_of_peaks": 0.4124697665, "histogram_number_of_zeroes": 0.0843761061, "histogram_tendency": 0.093261831, "histogram_variance": 0.4025858496, "histogram_width": 0.6982247103, "light_decelerations": 0.0, "mean_value_of_long_term_variability": 0.2342289273, "mean_value_of_short_term_variability": 0.3635269978, "percentage_of_time_with_abnormal_long_term_variability": 0.1582718911, "prolongued_decelerations": 0.0, "severe_decelerations": 0.0, "test col": 0.3635269978, "uterine_contractions": 0.0}, {"abnormal_short_term_variability": 0.003576094, "accelerations": 0.0, "baseline value": 0.4725201635, "fetal_health": 0.0330834684, "fetal_movement": 0.0, "histogram_max": 0.1328966249, "histogram_mean": 0.675096363, "histogram_median": 0.7706218281, "histogram_min": 0.1424701132, "histogram_mode": 1.0, "histogram_number_of_peaks": 0.0169524966, "histogram_number_of_zeroes": 0.0, "histogram_tendency": 0.0518563517, "histogram_variance": 0.0, "histogram_width": 0.0, "light_decelerations": 0.0, "mean_value_of_long_term_variability": 0.0, "mean_value_of_short_term_variability": 0.010523184, "percentage_of_time_with_abnormal_long_term_variability": 0.0, "prolongued_decelerations": 0.0, "severe_decelerations": 0.0, "test col": 0.010523184, "uterine_contractions": 0.0}, {"abnormal_short_term_variability": 0.1102120145, "accelerations": 0.0, "baseline value": 0.0341035432, "fetal_health": 0.0461326072, "fetal_movement": 0.0, "histogram_max": 0.1999548229, "histogram_mean": 0.0, "histogram_median": 0.0, "histogram_min": 0.27232448, "histogram_mode": 0.0, "histogram_number_of_peaks": 1.0, "histogram_number_of_zeroes": 0.0974076065, "histogram_tendency": 0.0, "histogram_variance": 0.253642096, "histogram_width": 0.327731029, "light_decelerations": 0.0, "mean_value_of_long_term_variability": 0.0569382345, "mean_value_of_short_term_variability": 0.2357970622, "percentage_of_time_with_abnormal_long_term_variability": 0.0447869281, "prolongued_decelerations": 0.0, "severe_decelerations": 0.0, "test col": 0.2357970622, "uterine_contractions": 0.0}, {"abnormal_short_term_variability": 0.0, "accelerations": 0.0, "baseline value": 0.0, "fetal_health": 0.0, "fetal_movement": 0.0, "histogram_max": 0.0, "histogram_mean": 0.0, "histogram_median": 0.0, "histogram_min": 0.0, "histogram_mode": 0.0, "histogram_number_of_peaks": 0.0, "histogram_number_of_zeroes": 1.0, "histogram_tendency": 0.0, "histogram_variance": 0.0, "histogram_width": 0.0, "light_decelerations": 0.0, "mean_value_of_long_term_variability": 0.0, "mean_value_of_short_term_variability": 0.0, "percentage_of_time_with_abnormal_long_term_variability": 0.0, "prolongued_decelerations": 0.0, "severe_decelerations": 0.0, "test col": 0.0, "uterine_contractions": 0.0}, {"abnormal_short_term_variability": 0.0, "accelerations": 0.0, "baseline value": 0.0422071928, "fetal_health": 0.0, "fetal_movement": 0.0, "histogram_max": 0.0746132442, "histogram_mean": 0.050856839, "histogram_median": 0.0730571242, "histogram_min": 0.1155537258, "histogram_mode": 0.0565168545, "histogram_number_of_peaks": 0.0134455798, "histogram_number_of_zeroes": 0.0, "histogram_tendency": 1.0, "histogram_variance": 0.0, "histogram_width": 0.2162945613, "light_decelerations": 0.0, "mean_value_of_long_term_variability": 0.0, "mean_value_of_short_term_variability": 0.0, "percentage_of_time_with_abnormal_long_term_variability": 0.0, "prolongued_decelerations": 0.0, "severe_decelerations": 0.0, "test col": 0.0, "uterine_contractions": 0.0}, {"abnormal_short_term_variability": 0.005060366, "accelerations": 0.0, "baseline value": 0.0, "fetal_health": 0.0, "fetal_movement": 0.0, "histogram_max": 0.099984152, "histogram_mean": 0.0, "histogram_median": 0.0, "histogram_min": 0.1639743099, "histogram_mode": 0.0, "histogram_number_of_peaks": 0.0408369312, "histogram_number_of_zeroes": 0.0, "histogram_tendency": 0.0, "histogram_variance": 1.0, "histogram_width": 0.2194804418, "light_decelerations": 0.0, "mean_value_of_long_term_variability": 0.0, "mean_value_of_short_term_variability": 0.2298104513, "percentage_of_time_with_abnormal_long_term_variability": 0.0, "prolongued_decelerations": 0.0, "severe_decelerations": 0.0, "test col": 0.2298104513, "uterine_contractions": 0.0}, {"abnormal_short_term_variability": 0.2197243607, "accelerations": 0.0, "baseline value": 0.0865459006, "fetal_health": 0.1443842513, "fetal_movement": 0.0, "histogram_max": 0.3586922483, "histogram_mean": 0.0038373379, "histogram_median": 0.0173363984, "histogram_min": 0.6513179484, "histogram_mode": 0.0587405321, "histogram_number_of_peaks": 0.4549033293, "histogram_number_of_zeroes": 0.1044175345, "histogram_tendency": 0.0319574841, "histogram_variance": 0.4830031364, "histogram_width": 1.0, "light_decelerations": 0.0, "mean_value_of_long_term_variability": 0.199376977, "mean_value_of_short_term_variability": 0.3994591378, "percentage_of_time_with_abnormal_long_term_variability": 0.2124018547, "prolongued_decelerations": 0.0, "severe_decelerations": 0.0, "test col": 0.3994591378, "uterine_contractions": 0.0}, {"abnormal_short_term_variability": 0.0, "accelerations": 0.0, "baseline value": 0.0, "fetal_health": 0.0, "fetal_movement": 0.0, "histogram_max": 0.0, "histogram_mean": 0.0, "histogram_median": 0.0, "histogram_min": 0.0, "histogram_mode": 0.0, "histogram_number_of_peaks": 0.0, "histogram_number_of_zeroes": 0.0, "histogram_tendency": 0.0, "histogram_variance": 0.0, "histogram_width": 0.0, "light_decelerations": 1.0, "mean_value_of_long_term_variability": 0.0, "mean_value_of_short_term_variability": 0.0, "percentage_of_time_with_abnormal_long_term_variability": 0.0, "prolongued_decelerations": 0.0, "severe_decelerations": 0.0, "test col": 0.0, "uterine_contractions": 0.0}, {"abnormal_short_term_variability": 0.0006375019, "accelerations": 0.0, "baseline value": 0.0, "fetal_health": 0.0370767862, "fetal_movement": 0.0, "histogram_max": 0.0, "histogram_mean": 0.0, "histogram_median": 0.0, "histogram_min": 0.0822068103, "histogram_mode": 0.0, "histogram_number_of_peaks": 0.0463451364, "histogram_number_of_zeroes": 0.0, "histogram_tendency": 0.0, "histogram_variance": 0.0061253968, "histogram_width": 0.0766275968, "light_decelerations": 0.0, "mean_value_of_long_term_variability": 1.0, "mean_value_of_short_term_variability": 0.0637390244, "percentage_of_time_with_abnormal_long_term_variability": 0.0687933469, "prolongued_decelerations": 0.0, "severe_decelerations": 0.0, "test col": 0.0637390244, "uterine_contractions": 0.0}, {"abnormal_short_term_variability": 0.3047169589, "accelerations": 0.0, "baseline value": 0.0869705183, "fetal_health": 0.0950043356, "fetal_movement": 0.0, "histogram_max": 0.1583100822, "histogram_mean": 0.0, "histogram_median": 0.0, "histogram_min": 0.3603588776, "histogram_mode": 0.0, "histogram_number_of_peaks": 0.1698975068, "histogram_number_of_zeroes": 0.0, "histogram_tendency": 0.0, "histogram_variance": 0.3625313732, "histogram_width": 0.3673895471, "light_decelerations": 0.0, "mean_value_of_long_term_variability": 0.0255375544, "mean_value_of_short_term_variability": 1.0, "percentage_of_time_with_abnormal_long_term_variability": 0.2124803871, "prolongued_decelerations": 0.0, "severe_decelerations": 0.0, "test col": 0.9982750432, "uterine_contractions": 0.0}, {"abnormal_short_term_variability": 0.3182646506, "accelerations": 0.0, "baseline value": 0.023976348, "fetal_health": 0.2275199031, "fetal_movement": 0.0, "histogram_max": 0.0189882459, "histogram_mean": 0.0, "histogram_median": 0.0, "histogram_min": 0.0692526943, "histogram_mode": 0.0, "histogram_number_of_peaks": 0.0, "histogram_number_of_zeroes": 0.0, "histogram_tendency": 0.0, "histogram_variance": 0.2983201126, "histogram_width": 0.2549939069, "light_decelerations": 0.0, "mean_value_of_long_term_variability": 0.0988384576, "mean_value_of_short_term_variability": 0.1317691864, "percentage_of_time_with_abnormal_long_term_variability": 1.0, "prolongued_decelerations": 0.0, "severe_decelerations": 0.0, "test col": 0.1317691864, "uterine_contractions": 0.0}, {"abnormal_short_term_variability": 0.0, "accelerations": 0.0, "baseline value": 0.0, "fetal_health": 0.0, "fetal_movement": 0.0, "histogram_max": 0.0, "histogram_mean": 0.0, "histogram_median": 0.0, "histogram_min": 0.0, "histogram_mode": 0.0, "histogram_number_of_peaks": 0.0, "histogram_number_of_zeroes": 0.0, "histogram_tendency": 0.0, "histogram_variance": 0.0, "histogram_width": 0.0, "light_decelerations": 0.0, "mean_value_of_long_term_variability": 0.0, "mean_value_of_short_term_variability": 0.0, "percentage_of_time_with_abnormal_long_term_variability": 0.0, "prolongued_decelerations": 1.0, "severe_decelerations": 0.0, "test col": 0.0, "uterine_contractions": 0.0}, {"abnormal_short_term_variability": 0.0, "accelerations": 0.0, "baseline value": 0.0, "fetal_health": 0.0, "fetal_movement": 0.0, "histogram_max": 0.0, "histogram_mean": 0.0, "histogram_median": 0.0, "histogram_min": 0.0, "histogram_mode": 0.0, "histogram_number_of_peaks": 0.0, "histogram_number_of_zeroes": 0.0, "histogram_tendency": 0.0, "histogram_variance": 0.0, "histogram_width": 0.0, "light_decelerations": 0.0, "mean_value_of_long_term_variability": 0.0, "mean_value_of_short_term_variability": 0.0, "percentage_of_time_with_abnormal_long_term_variability": 0.0, "prolongued_decelerations": 0.0, "severe_decelerations": 1.0, "test col": 0.0, "uterine_contractions": 0.0}, {"abnormal_short_term_variability": 0.3047169589, "accelerations": 0.0, "baseline value": 0.0869705183, "fetal_health": 0.0950043356, "fetal_movement": 0.0, "histogram_max": 0.1583100822, "histogram_mean": 0.0, "histogram_median": 0.0, "histogram_min": 0.3603588776, "histogram_mode": 0.0, "histogram_number_of_peaks": 0.1698975068, "histogram_number_of_zeroes": 0.0, "histogram_tendency": 0.0, "histogram_variance": 0.3625313732, "histogram_width": 0.3673895471, "light_decelerations": 0.0, "mean_value_of_long_term_variability": 0.0255375544, "mean_value_of_short_term_variability": 0.9982750432, "percentage_of_time_with_abnormal_long_term_variability": 0.2124803871, "prolongued_decelerations": 0.0, "severe_decelerations": 0.0, "test col": 1.0, "uterine_contractions": 0.0}, {"abnormal_short_term_variability": 0.0, "accelerations": 0.0, "baseline value": 0.0, "fetal_health": 0.0, "fetal_movement": 0.0, "histogram_max": 0.0, "histogram_mean": 0.0, "histogram_median": 0.0, "histogram_min": 0.0, "histogram_mode": 0.0, "histogram_number_of_peaks": 0.0, "histogram_number_of_zeroes": 0.0, "histogram_tendency": 0.0, "histogram_variance": 0.0, "histogram_width": 0.0, "light_decelerations": 0.0, "mean_value_of_long_term_variability": 0.0, "mean_value_of_short_term_variability": 0.0, "percentage_of_time_with_abnormal_long_term_variability": 0.0, "prolongued_decelerations": 0.0, "severe_decelerations": 0.0, "test col": 0.0, "uterine_contractions": 1.0}], "HasBasicStats": 1, "functionsApplied": [{"functionName": "replaceBy", "applyOn": [{"columnName": "percentage_of_time_with_abnormal_long_term_variability", "type": "numeric", "min": "0.0", "max": "91.0", "mean": "15.5"}], "functionsData": [{"replaceType": "none", "toReplace": "", "asNewColumn": 0, "newColumnName": "", "replaceValueType": "byValue", "ReplaceWith": "1"}]}, {"functionName": "editname", "applyOn": [{"columnName": "mean_value_of_short_term_variability", "type": "real", "min": "0.0", "max": "7.0", "mean": "1.1"}], "functionsData": [{"oldName": "mean_value_of_short_term_variability", "newName": "test col", "asNewColumn": 1, "column": {"columnName": "mean_value_of_short_term_variability", "type": "real", "min": "0.0", "max": "7.0", "mean": "1.1"}}]}], "functionChanges": [{"columnName": "percentage_of_time_with_abnormal_long_term_variability", "functionName": "Replace Outliers", "Type": "numeric", "Parameters": [{"replaceType": "none", "toReplace": "", "asNewColumn": 0, "newColumnName": "", "replaceValueType": "byValue", "ReplaceWith": "1"}]}, {"columnName": "mean_value_of_short_term_variability", "functionName": "Edit Column Name", "Type": "real", "Parameters": [{"oldName": "mean_value_of_short_term_variability", "newName": "test col", "asNewColumn": 1, "column": {"columnName": "mean_value_of_short_term_variability", "type": "real", "min": "0.0", "max": "7.0", "mean": "1.1"}}]}], "fileheader": [{"field": "baseline value", "alias": "baseline value", "generated": 0, "position": 1, "type": "numeric"}, {"field": "accelerations", "alias": "accelerations", "generated": 0, "position": 2, "type": "real"}, {"field": "fetal_movement", "alias": "fetal_movement", "generated": 0, "position": 3, "type": "real"}, {"field": "uterine_contractions", "alias": "uterine_contractions", "generated": 0, "position": 4, "type": "real"}, {"field": "light_decelerations", "alias": "light_decelerations", "generated": 0, "position": 5, "type": "real"}, {"field": "severe_decelerations", "alias": "severe_decelerations", "generated": 0, "position": 6, "type": "numeric"}, {"field": "prolongued_decelerations", "alias": "prolongued_decelerations", "generated": 0, "position": 7, "type": "real"}, {"field": "abnormal_short_term_variability", "alias": "abnormal_short_term_variability", "generated": 0, "position": 8, "type": "numeric"}, {"field": "mean_value_of_short_term_variability", "alias": "mean_value_of_short_term_variability", "generated": 0, "position": 9, "type": "real"}, {"field": "percentage_of_time_with_abnormal_long_term_variability", "alias": "percentage_of_time_with_abnormal_long_term_variability", "generated": 0, "position": 10, "type": "numeric"}, {"field": "mean_value_of_long_term_variability", "alias": "mean_value_of_long_term_variability", "generated": 0, "position": 11, "type": "real"}, {"field": "histogram_width", "alias": "histogram_width", "generated": 0, "position": 12, "type": "numeric"}, {"field": "histogram_min", "alias": "histogram_min", "generated": 0, "position": 13, "type": "numeric"}, {"field": "histogram_max", "alias": "histogram_max", "generated": 0, "position": 14, "type": "numeric"}, {"field": "histogram_number_of_peaks", "alias": "histogram_number_of_peaks", "generated": 0, "position": 15, "type": "numeric"}, {"field": "histogram_number_of_zeroes", "alias": "histogram_number_of_zeroes", "generated": 0, "position": 16, "type": "numeric"}, {"field": "histogram_mode", "alias": "histogram_mode", "generated": 0, "position": 17, "type": "numeric"}, {"field": "histogram_mean", "alias": "histogram_mean", "generated": 0, "position": 18, "type": "numeric"}, {"field": "histogram_median", "alias": "histogram_median", "generated": 0, "position": 19, "type": "numeric"}, {"field": "histogram_variance", "alias": "histogram_variance", "generated": 0, "position": 20, "type": "numeric"}, {"field": "histogram_tendency", "alias": "histogram_tendency", "generated": 0, "position": 21, "type": "numeric"}, {"field": "fetal_health", "alias": "fetal_health", "generated": 0, "position": 22, "type": "numeric"}, {"field": "test col", "alias": "test col", "generated": 1, "position": 23, "type": "real"}]}))
	#operationPostExecutionHook(datapreparation)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run dataHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionClassification(datapreparation, ["baseline value", "accelerations", "fetal_movement", "uterine_contractions", "light_decelerations", "severe_decelerations", "prolongued_decelerations", "abnormal_short_term_variability", "percentage_of_time_with_abnormal_long_term_variability", "mean_value_of_long_term_variability", "histogram_width", "histogram_min", "histogram_max", "histogram_number_of_peaks", "histogram_number_of_zeroes", "histogram_mode", "histogram_mean", "histogram_median", "histogram_variance", "histogram_tendency", "test col"], "fetal_health")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    Accuracy = np.round((100 * sklearn.metrics.accuracy_score(y_true=y_test, y_pred=y_predicted)), 1)
    F1= np.round(
            (100 * sklearn.metrics.f1_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Precision= np.round((
                100 * sklearn.metrics.precision_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Recall = np.round((
                100 * sklearn.metrics.recall_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    display(" Accuracy of Prediction on test data    : %s"%Accuracy)
    display(" F1 score of Prediction on test data    : %s"%F1)
    display(" Precision of Prediction on test data   : %s"%Precision)
    display(" Recall of Prediction on test data      : %s"%Recall)
    display(df.head())
except Exception as ex:
    logging.error(ex)

